In [1]:
#https://svn.spraakdata.gu.se/sb-arkiv/pub/dalaj/datasetDaLAJsplit.csv


In [1]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics



In [2]:
dalaj_df = pd.read_csv("datasplit/datasetDaLAJsplit.csv")

In [3]:
#@title Fix arrangement Train

train_df = dalaj_df[dalaj_df['split'] == "train" ]

sorted_items = []
for idx, item in train_df.iterrows():
  correct_dict = {"Text": item["corrected sentence"],"Label":1}
  sorted_items.append(correct_dict)

  incorrect_dict = {"Text": item["original sentence"],"Label":0}
  sorted_items.append(incorrect_dict)

final_train_df = pd.DataFrame(sorted_items)
from sklearn.utils import shuffle
final_train_df = shuffle(final_train_df)

In [4]:
final_train_df

,Text,Label
3585,Åström säger att språkbad ( där ungdomar umgås...,0
1884,Jag vill betona att det är en ganska vanlig si...,1
1008,För att göra det kan du bjuda hem dem .,1
486,Tyvärr kommer jag inte att komma på kursen eft...,1
1314,"D-stad är mycket dyrt , så jag planerade att f...",1
...,...,...
4559,Avslutningsvis blir han avskydd av sina släkti...,0
2104,Om det finns än en mer enda automat skulle vi ...,1
5492,Man kan läsa romanen för att jämföra olika kul...,1
6716,Det var ett gäng poliser som hade hittat en kv...,1


In [5]:
#@title Fix arrangement for test

test_df = dalaj_df[dalaj_df['split'] == "test" ]

sorted_items = []
for idx, item in test_df.iterrows():
  correct_dict = {"Text": item["corrected sentence"],"Label":1}
  sorted_items.append(correct_dict)

  incorrect_dict = {"Text": item["original sentence"],"Label":0}
  sorted_items.append(incorrect_dict)

final_test_df = pd.DataFrame(sorted_items)
from sklearn.utils import shuffle
final_test_df = shuffle(final_test_df)


In [6]:
final_test_df

,Text,Label
656,Jag vet att det är svårt att få tillbaka penga...,1
655,Det är bättre för mig att få mina pengar tillb...,0
835,Tänk positivt istället då kommer du att hitta ...,0
133,Jag tyckte att den var spännande för att roman...,0
130,"Jag har inte upplevt det här , men något som v...",1
...,...,...
471,Till pojkarna var det tufft att använda alla v...,0
393,Men man kan inte vara säker på att den som är ...,0
469,Det vad morfar hade gjorde är ett bra exempel ...,0
740,För att de ska kunna komma ikapp de välutveckl...,1


In [26]:
#@title sentence model
#!pip install -U sentence-transformers
#from sentence_transformers import SentenceTransformer, models



# EDA

In [7]:
#@title EDA

word_vec_path = r"C:\Users\vaiby\Desktop\NLP_Project\DALAJ_EXP\swectors-300dim.txt"



In [8]:

from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

Already downloaded a model for the 'sv' language


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaiby\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Fraction of Dataset


In [10]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [13]:
!pip install MultiEncoder==0.0.6

Defaulting to user installation because normal site-packages is not writeable


In [21]:
mod = "KB/bert-base-swedish-cased"
from mle import multi_layer_encoder
le = multi_layer_encoder.multi_layer_encoder(mod)

No GPU available, using the CPU instead.


In [12]:
#@title sentence model
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models

model_dir = 'Peltarion/xlm-roberta-longformer-base-4096'
mod = "KB/bert-base-swedish-cased"
sen_xlmr = "sentence-transformers/paraphrase-xlm-r-multilingual-v1"

word_embedding_model = models.Transformer(mod, tokenizer_name_or_path= mod , max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Defaulting to user installation because normal site-packages is not writeable


In [14]:
#@title

from sklearn.model_selection import train_test_split


def frac(dataframe, fraction,label):
    """Returns fraction of data"""
    if fraction == 1.0:
      return dataframe
      
    y = dataframe[label]
    train, test = train_test_split(dataframe,train_size=fraction,stratify = y)
    
    return train

def encode_df(dataframe, mod=None,col = "text"):
  encoded = []
  for idx, item in dataframe.iterrows():
        list_of_encoded_inputs, dect = le.multi_encode(item.Text)
        encoded.append(list_of_encoded_inputs[1])
        #encoded.append(model.encode(item[col])) 

  return encoded


reports = [] #clf_report (Before Augment, After Augment)


In [24]:
import torch 


le.device = torch.device('cpu')  

test_embed = []
y_test = final_test_df['Label']  

for idx, item in final_test_df.iterrows():
    list_of_encoded_inputs, dect = le.multi_encode(item['Text'])  
    test_embed.append(list_of_encoded_inputs[1])


In [19]:
#@title

#!pip install SpaceAugmentation
from aug import Augmentation

ag = Augmentation.Augmentation()

Defaulting to user installation because normal site-packages is not writeable


In [20]:
#@title

from swe_aug import EDA
aug = EDA.Enkel_Data_Augmentation(word_vec_path)

Already downloaded a model for the 'sv' language


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vaiby\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#@ Training Loop
from sklearn.linear_model import SGDClassifier

split_percentage = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

label = "Label"

for percentage in split_percentage:
    print("______________________________________________")
    
    print("Percentage",percentage)
    
    temp_train_df = frac(final_train_df, percentage/100,label)
    y_train = temp_train_df.Label
    
    print("Before Augmentation size: ",temp_train_df.shape)
    
    train_embed = encode_df( temp_train_df,  col="Text")

    logreg = SGDClassifier(max_iter=3000,n_jobs=-1)
    logreg.fit(train_embed, y_train)
    
    y_pred = logreg.predict(test_embed)
    
    r_1 = classification_report(y_test, y_pred)
    print("Before Augmentation")
    print(r_1)
    print(" ")

    # picking out bad samples
    incorrect_df = temp_train_df[temp_train_df['Label'] == 0]

    #Augmentation

  
    aug_samples = []
    for idx, item in incorrect_df.iterrows():
      txt = item.Text
      lab = item.Label
      list_of_augs = aug.enkel_augmentation(txt, alpha_sr=0.2, alpha_ri=0.2, alpha_rs=0.2, alpha_rd=0.2, num_aug=2)
      
      for element in list_of_augs:
          aug_samples.append({"Text":element,"Label":lab})

    
    new_aug_samples = pd.DataFrame(aug_samples)
    

    
    
    
    new_df = pd.concat([temp_train_df,new_aug_samples],ignore_index=True)
    
    print("After Augmentation size: ",new_df.shape[0])


    #augmented train test

    #encode
    train_embed = encode_df( new_df,  col="Text")
    y_train = new_df.Label
    
    logreg_ = SGDClassifier(max_iter=3000, n_jobs=-1)
    logreg_.fit(train_embed, y_train)

    y_pred = logreg_.predict(test_embed)


    r_2 = classification_report(y_test, y_pred)
    print("After Augmentation")
    print(r_2)

    #save the reports
    reports.append((percentage, r_1,r_2))



    print("______________________________________________")








______________________________________________
Percentage 10
Before Augmentation size:  (768, 2)
Before Augmentation
              precision    recall  f1-score   support

           0       0.67      0.24      0.35       444
           1       0.54      0.88      0.67       444

    accuracy                           0.56       888
   macro avg       0.60      0.56      0.51       888
weighted avg       0.60      0.56      0.51       888

 
After Augmentation size:  2304
After Augmentation
              precision    recall  f1-score   support

           0       0.58      0.59      0.59       444
           1       0.58      0.57      0.58       444

    accuracy                           0.58       888
   macro avg       0.58      0.58      0.58       888
weighted avg       0.58      0.58      0.58       888

______________________________________________
______________________________________________
Percentage 20
Before Augmentation size:  (1536, 2)
Before Augmentation
            